## NDVI after the Las Conchas fire in New Mexico

This notebook will be used to download NDVI data using the NASA appeeaars api.

In [ ]:
# Import Libraries

import pathlib
import os
import earthpy

import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import xarray as xr
import holoviews as hv

import hvplot.pandas

In [ ]:
# Make a project and a directory

project = earthpy.Project('Las Conchas Vegetation', dirname='fire-veg-data')